In [2]:
pwd

'c:\\Users\\praka\\Acads\\Work\\AI_Agents\\Text-to-Pyspark-agent\\ingestion'

In [3]:
cd ..

c:\Users\praka\Acads\Work\AI_Agents\Text-to-Pyspark-agent


In [1]:
1+2

3

In [ ]:
# TODO:in metadata, once a column is added, add other columns in this table as another line of metadata
# if retivral fails to give some cols, maybe that table has some other col that wasnt retrived but its useful
# can use cosine sim or something to pick only 20 cols etc


# TODO: add counts and column counts of tables in metadata, will be later used for determining hard persist / persist

In [18]:
text = json.dumps(data, indent=4)
text

'{\n    "6b5f03f590cf5a09a7f371d46bc928c2": {\n        "column_name": "Actor_Gender",\n        "column_desc": "The gender of the actor (e.g., Male, Female, Other).",\n        "column_type": "string",\n        "is_column_nullable": "false",\n        "table_name": "actors_EN",\n        "table_desc": "Stores demographic and biographical information about actors, including their names, birth dates, nationality, and a summary of their career.",\n        "table_path": "data/actors_EN.csv",\n        "table_extenstion": "csv",\n        "granularity_key": "ActorID",\n        "example_values": [\n            "Other",\n            "Male",\n            "Female"\n        ],\n        "data_distribution": [\n            [\n                "unique_count",\n                3\n            ],\n            [\n                "top_5_values",\n                [\n                    [\n                        "Female",\n                        241\n                    ],\n                    [\n             

In [19]:
import json
from typing import Dict, List
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings  # This moved from langchain.embeddings
from langchain_core.documents import Document # This moved from langchain.schema
from langchain_huggingface.embeddings.huggingface import HuggingFaceEmbeddings


from const import EMBEDDING_CHUNK_SIZE
# --- Load JSON ---
with open("data/column_metadata.json", "r") as f:
    data = json.load(f)


def split_on_size_and_key_boundary(json_obj: Dict, max_chunk_size: int = 2000) -> List[str]:
    """
    Splits a JSON object into smaller string chunks. The primary goal is to 
    force a split between top-level keys (hash IDs) to ensure each Chroma 
    document contains data from only one column/hash ID, while also respecting 
    the max_chunk_size.

    Args:
        json_obj (dict): The JSON object to split.
        max_chunk_size (int): Maximum size of each chunk in characters (default 2000).

    Returns:
        List[str]: List of JSON string chunks, primarily split on top-level key boundaries.
    """
    # 1. Convert the entire JSON object to a formatted string
    # Using indent=4 is crucial as it creates predictable newline/space patterns for splitting.
    text = json.dumps(json_obj, indent=4)
    
    # 2. Check if the total size is within the limit
    if len(text) <= max_chunk_size:
        return [text]
    
    # 3. Use the RecursiveCharacterTextSplitter
    # The order of separators is critical: the splitter tries them in order.
    # By putting the key boundary separator first, we maximize the chance of 
    # splitting cleanly between hash IDs.
    
    # Primary Splitter: The separator for top-level dictionary entries (hash IDs)
    # The pattern ',\n    ' marks the end of one key's block and the start of the next 
    # when indent=4 is used.
    KEY_BOUNDARY_SEPARATOR = ",\n    "
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chunk_size,
        chunk_overlap=0, # Use zero overlap to cleanly separate hash IDs
        separators=[
            KEY_BOUNDARY_SEPARATOR, 
            "\n\n",   # Secondary split: Double newline
            "\n}",    # Tertiary split: Before closing curly bracket (within a hash ID block)
            "\n]",    # Tertiary split: Before closing square bracket
            " ",
            ""
        ]
    )
    
    chunks = splitter.split_text(text)
    return chunks


# --- Create LangChain Documents ---
docs = []
for hash_id, content in data.items():
    splits = split_on_size_and_key_boundary(content, max_chunk_size=EMBEDDING_CHUNK_SIZE)
    for idx, chunk in enumerate(splits):
        docs.append(
            Document(
                page_content=chunk,
                metadata={
                    "hash_id": hash_id,
                    "part": idx,
                    "table_name": content.get("table_name", ""),
                    "column_name": content.get("column_name", "")
                }
            )
        )



In [22]:
docs[12]

Document(metadata={'hash_id': '8cab69ed2d9d46fbd07e3af33897bb24', 'part': 0, 'table_name': 'movies_EN', 'column_name': 'MovieID'}, page_content='{\n    "column_name": "MovieID",\n    "column_desc": "A unique identifier for each movie.",\n    "column_type": "int",\n    "is_column_nullable": "false",\n    "table_name": "movies_EN",\n    "table_desc": "Stores information about movies, including their title, release year, duration, language, country, budget, box office, and a short description.",\n    "table_path": "data/movies_EN.csv",\n    "table_extenstion": "csv",\n    "granularity_key": "MovieID",\n    "example_values": [\n        "1",\n        "2",\n        "3",\n        "4",\n        "5"\n    ],\n    "data_distribution": [\n        [\n            "count",\n            300.0\n        ],\n        [\n            "min",\n            1.0\n        ],\n        [\n            "25%",\n            75.75\n        ],\n        [\n            "50%",\n            150.5\n        ],\n        [\n    

In [23]:
# --- Create embeddings & store in Chroma ---
# embedding = OpenAIEmbeddings()  # or HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", 
                                  cache_folder="./models")
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory="chroma_db/metadata_huggingface"
)

# vectorstore.persist()
print(f"✅ Ingested {len(docs)} documents into Chroma DB.")

✅ Ingested 34 documents into Chroma DB.


In [24]:
# --- 1. Test the retriever (in the same script) ---
print("\n--- Testing Retriever ---")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Get the top 3 most relevant documents
)

query = "What is the biography of the actor named Paul Simon?"
retrieved_docs = retriever.invoke(query)

print(f"Query: {query}\n")
for i, doc in enumerate(retrieved_docs):
    print(f"--- Result {i+1} ---")
    print(doc.page_content)
    print(doc.metadata)
    print("------------------\n")


--- Testing Retriever ---
Query: What is the biography of the actor named Paul Simon?

--- Result 1 ---
{
    "column_desc": "A short biography or career summary for the actor."
}
{'part': 1, 'column_name': 'Biography', 'hash_id': '0037d45c433df22aa782f1c63e4d2f3a', 'table_name': 'actors_EN'}
------------------

--- Result 2 ---
{
    "table_desc": "Stores demographic and biographical information about actors, including their names, birth dates, nationality, and a summary of their career."
}
{'hash_id': '0037d45c433df22aa782f1c63e4d2f3a', 'part': 5, 'table_name': 'actors_EN', 'column_name': 'Biography'}
------------------

--- Result 3 ---
{
    "column_name": "Biography",
    "column_desc": "A short biography or career summary for the actor.",
    "column_type": "string",
    "is_column_nullable": "false",
    "table_name": "actors_EN",
    "table_desc": "Stores demographic and biographical information about actors, including their names, birth dates, nationality, and a summary of th